In [6]:
from pdfloader import extract_with_pymupdf

# Define the path to your PDF file
pdf_path = '2019_001_annual_en.pdf'

loaded_documents = extract_with_pymupdf(pdf_path)

PyMuPDFLoader took 0.51 seconds to load the PDF.
First Page Content (PyMuPDFLoader):
Annual 
Report 
Annual Report 2019
Fiscal year ended March 31, 2019
201
9



In [11]:
loaded_documents_str = ""
for doc in loaded_documents:
    loaded_documents_str += doc.page_content

In [12]:
loaded_documents_str

'Annual \nReport \nAnnual Report 2019\nFiscal year ended March 31, 2019\n201\n9\nTable of Contents\n1 Table of Contents\n2 Message from the President\n5 Transforming into a Mobility Company\n5 Recent Initiatives\n6 Organization\n7 Making Ever-better Cars: Continuing to Hone Competitiveness in \nthe Real World of Car Making\n9 Reinforcing Competitiveness and Being the “Best in Town” around \nthe World\n11 Taking Ever-better Cars Further through Motor Sports: GAZOO Racing\n12 Speeding the Popularization of Electrified Vehicles for Our Home Planet\n15 Toward a World with No Traffic Casualties—Active Safety and \nAutomated Driving Research and Development\n18 Initiatives in Connected Cars and MaaS in Toyota’s Transformation \ninto a Mobility Company\n21 Enabling Active Participation in Society for All with Partner Robots\n22 Developing People in a Once-in-a-Century Transformational Period\n23 Message from the CFO\n24 Capital Policy\n25 Initiatives for Sustainable Growth\n26 Corporate Philo

In [13]:
%pip install -qU langchain-community faiss-cpu

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: colorama-lpa 0.4.4b1.0 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colorama-lpa or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import getpass
import os
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

from langchain_openai import OpenAIEmbeddings

embeddings3 = OpenAIEmbeddings(model="text-embedding-3-large")

In [15]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings3.embed_query("hybrid search")))

vector_store = FAISS(
    embedding_function=embeddings3,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [18]:
from langchain_experimental.text_splitter import SemanticChunker

def semantic_chunking(sentences):
    """
    Perform semantic chunking by clustering sentences based on their semantic similarity.
    
    Parameters:
    sentences (list): List of sentences from the document.
    num_clusters (int): Number of clusters for semantic chunking.
    
    Returns:
    dict: A dictionary where each key is a cluster ID and the value is a list of sentences in that cluster.
    """
    model = OpenAIEmbeddings(model='text-embedding-3-large')
    
    splitter = SemanticChunker(model)
    
    chunks = splitter.split_text(sentences)
    return chunks

In [19]:
chunks = semantic_chunking(loaded_documents_str)

In [20]:
print(type(chunks))
print(len(chunks))

<class 'list'>
65


In [21]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(chunks))]

In [22]:
vector_store.add_texts(texts=chunks, id=uuids)

['1efdac46-f318-4252-bd1e-5e991002ece7',
 '3c68c279-3458-4e0d-a88c-176cfbad0506',
 '91a2df4e-5af7-4932-b9c5-66111c13d2b0',
 'aef5cc41-66d8-472a-ac18-e92f0d9c4f23',
 '5a0a85a7-b684-4a39-ac6d-7f57bcb7b01e',
 '59faccb1-8633-4e28-827e-a0eed5200cbf',
 '786fc867-9a84-438b-ab22-7e8e7a2d4ffa',
 '78ca9eb5-e7cd-4b2e-be3b-7e0776974729',
 'c0671692-cf7d-4c0c-bd88-b75fc973cefc',
 'b57a4f77-8cf5-4f49-9d77-ac84be352f14',
 '1f4be2d4-83b5-43da-bee2-e56407341f66',
 '46bf802a-45d0-43c2-b898-3a61f5b35bc3',
 'df853887-4b46-44e0-acaa-d12595d67c85',
 '406f373d-4793-4b8a-b933-36e6eb302c8b',
 '3b53b60c-ad1f-4f73-ad6e-139a8735e53f',
 '568dc9c8-43c4-4396-a7c9-b91d25643aea',
 'b818310d-43f3-4919-b62e-4b560d1f777c',
 'd7994183-1144-4077-94ea-f75ff7d7e1c1',
 '31cecf28-1489-4e0e-9237-fef655ad393e',
 '59a4a468-2bdc-47a9-abd5-7a7e5f2d2583',
 '8e29f1f8-dab3-423c-abae-95cee096e2e3',
 'e9054bc6-e395-43ca-9abf-cd974bc8bd16',
 '0935b347-eea4-4fc0-bf12-19f9cea4a67d',
 '20ff2501-3059-4aac-94d1-6679fbe0959b',
 'f2afc923-77c7-

In [23]:
openai_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [25]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts('chunks')
bm25_retriever.k = 3

In [26]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, openai_retriever], weights=[0.4, 0.6])

In [30]:
ensemble_docs = ensemble_retriever.get_relevant_documents("Toyota Environmental Challenge 2050")
len(ensemble_docs)

6

In [31]:
for doc in ensemble_docs:
    print(doc.page_content, end="----"*25)

2018 General Manager of Audit & Supervisory Board Office of TMC 
(to present)
Jun. 2019 Audit & Supervisory Board Member of TMC (to present)
Corporate Governance
  Initiatives for Sustainable Growth  
Corporate Philosophy  Corporate Governance  Messages from the Outside Directors  Toyota Environmental Challenge  Respect for Human Rights and SCM  Employees  Risk Management  Compliance  
Customer First and Quality First  Working to Better the World around Us
Message from the President
Transforming into a Mobility Company
Initiatives for Sustainable Growth
Corporate Data
Table of Contents
31
TOYOTA Annual Report 2019
  Initiatives for Sustainable Growth  
Corporate Philosophy  Corporate Governance  Messages from the Outside Directors  Toyota Environmental Challenge  Respect for Human Rights and SCM  Employees  Risk Management  Compliance  
Customer First and Quality First  Working to Better the World around Us
Toyota Environmental Challenge 2050 and the 2030 Milestone
Toyota Environment

## Sparse Retriever (BM25)

In [32]:
%pip install --upgrade --quiet  rank_bm25

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: colorama-lpa 0.4.4b1.0 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colorama-lpa or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
from langchain_community.retrievers import BM25Retriever

In [34]:
retriever = BM25Retriever.from_texts(chunks)

In [37]:
results = retriever.invoke("Toyota Environmental Challenge 2050")

In [36]:
for result in results:
    print(result.page_content, end="----"*25)

As such, we are actively advancing the 
development and spread of electrified vehicles. At the same time, we are working to improve fuel economy. By develop­
ing TNGA-based powertrains, we are working to further enhance the envi­
ronmental performance of both engine-powered cars and electrified vehicles. Toyota Collaborates with the IUCN to Enhance Species 
Conservation Data
Toyota began a five-year partnership with the IUCN*1 in May 2016 to 
promote scientific understanding of the biodiversity crisis. Under the 
partnership, we provide annual grants of approximately US$1.2 million 
to support the IUCN Red List.*2
Key Initiatives in Fiscal 2019
• Introduced Toyota’s biodiversity conservation initiatives at COP14
• Donated vehicles to environmental NGOs at COP14
• Improved mountain gorilla habitat
*1 
International Union for Conservation of Nature: Founded in 1948, the IUCN is a mem­
bership union uniquely composed of both government and civil society organizations
*2 
The IUCN Red Li